In [6]:
import numpy as np

from deepymod.DeepMoD import DeepMoD
from deepymod.library_functions import library_1D
from deepymod.utilities import library_matrix_mat, print_PDE

import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')

np.random.seed(42)  # setting seed for randomisation

In [7]:
noise_level = 0.05
number_of_samples = 1000
layers = [2, 20, 20, 20, 20, 20, 1]
l1 = 10**-6

In [8]:
# %%Importing and preparing data
data = np.load('data/burgers.npy').item()

X = np.transpose((data['x'].flatten(), data['t'].flatten()))
y = np.real(data['u']).reshape((data['u'].size, 1))
y_noisy = y + noise_level * np.std(y) * np.random.randn(y.size, 1)

idx = np.random.permutation(y.size)
X_train = X[idx, :][:number_of_samples]
y_train = y[idx, :][:number_of_samples]

In [9]:
config = {'layers': layers, 'lambda': l1}
library_config = {'total_terms': 12, 'deriv_order': 3, 'poly_order': 2}
train_opts = {'max_iterations': 10000, 'grad_tol':10**-7, 'learning_rate': 0.002, 'beta1': 0.99, 'beta2': 0.999, 'epsilon': 10**-8}
output_opts = {'output_directory': 'burgers_output/', 'X_predict': X}

In [10]:
sparse_vectors, denoised = DeepMoD(X_train, y_train, config, library_1D, library_config, train_opts, output_opts)

W0405 12:01:16.570682 4383253952 deprecation.py:323] From /Users/gert-janboth/Documents/PhD/Software/DeePyMoD/src/deepymod/graphs.py:22: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
W0405 12:01:16.579741 4383253952 deprecation.py:323] From /Users/gert-janboth/Documents/PhD/Software/DeePyMoD/src/deepymod/graphs.py:29: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0405 12:01:18.539993 4383253952 deprecation.py:506] From /anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/training/slot_creator.py:187: calling Zeros

Epoch | Total loss | Loss gradient | MSE | PI | L1 
0 [0.20951772, 0.00689468, 0.04203936, 0.16747703, 1.3283852e-06]
500 [0.0023471161, 0.0007668776, 0.0013462023, 0.0009888678, 1.2045854e-05]
1000 [0.00077232387, 0.00014665404, 0.00039132376, 0.00037083827, 1.0161846e-05]
1500 [0.00037347092, 8.112131e-05, 0.00015941051, 0.00020560312, 8.457281e-06]
2000 [0.00019763783, 8.8902154e-05, 7.380601e-05, 0.00011727144, 6.560381e-06]
2500 [0.00012609403, 7.362064e-05, 4.3673597e-05, 7.7159246e-05, 5.261194e-06]
3000 [7.954376e-05, 5.4599554e-05, 2.6628355e-05, 4.784828e-05, 5.067121e-06]
3500 [5.2657877e-05, 3.6878617e-05, 1.9486133e-05, 2.8386456e-05, 4.785289e-06]
4000 [3.5811947e-05, 2.9663634e-05, 1.3952622e-05, 1.7449696e-05, 4.4096264e-06]
4500 [2.3359451e-05, 2.3950146e-05, 8.844946e-06, 1.0605192e-05, 3.9093147e-06]
5000 [1.4849805e-05, 1.8475115e-05, 5.3093263e-06, 6.2059094e-06, 3.3345684e-06]
5500 [9.510925e-06, 1.4998694e-05, 2.9867483e-06, 3.6885185e-06, 2.8356585e-06]
6000 [5.

W0405 12:03:44.251443 4383253952 deprecation.py:323] From /Users/gert-janboth/Documents/PhD/Software/DeePyMoD/src/deepymod/graphs.py:69: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [11]:
u = ['1', 'u', 'uˆ2']
du = ['1', 'u_{x}', 'u_{xx}', 'u_{xxx}']
coeffs_list = library_matrix_mat(u, du)

print('Inferred equation:')
(print_PDE(coeff, coeffs_list, PDE_term='u_t') for coeff in sparse_vectors)

Inferred equation:
u_t = 0.100u_{xx} - 0.997uu_{x}


[None]